# 12. Special Methods for Sequences

> Don’t check whether it is-a duck: check whether it quacks-like-a duck, walks-like-a duck, etc., etc., depending on exactly what subset of duck-like behavior you need to play your language-games with. 
>
> Alex Martelli

## Vector Take #1: Vector2d Compatible

The first version of `Vector` should be as compatible as possible with our earlier `Vector2d` class.

However, by design, the Vector constructor is not compatible with the Vector2d constructor. We could make `Vector(3, 4)` and `Vector(3, 4, 5)` work, by taking arbitrary arguments with *args in `__init__`, but the best practice for a sequence con‐ structor is to take the data as an iterable argument in the constructor, like all built-in sequence types do.

In [1]:
from array import array
import reprlib
import math

In [2]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1] 
        return f'Vector({components})'
    
    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(*self)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode) 
        return cls(memv)

In [3]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [4]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [5]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

## Protocols and Duck Typing

As early as Chapter 1, we saw that you don’t need to inherit from any special class to create a fully functional sequence type in Python; you just need to implement the methods that fulfill the sequence protocol. But what kind of protocol are we talking about?

In the context of object-oriented programming, a protocol is an informal interface, defined only in documentation and not in code. For example, the sequence protocol in Python entails just the `__len__` and `__getitem__` methods. Any class `Spam` that implements those methods with the standard signature and semantics can be used anywhere a sequence is expected. Whether `Spam` is a subclass of this or that is irrelevant; all that matters is that it provides the necessary methods.

In [6]:
def __len__(self):
    return len(self._components)

def __getitem__(self, index):
    return self._components[index]

Vector.__len__ = __len__
Vector.__getitem__ = __getitem__

In [7]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [8]:
v1[0], v1[-1]

(3.0, 5.0)

In [10]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

### How Slicing Works

In [11]:
class MySeq:
    def __getitem__(self, index):
        return index

In [12]:
s = MySeq()
s[1]

1

In [13]:
s[1:4]

slice(1, 4, None)

In [14]:
s[1:4:2]

slice(1, 4, 2)

In [15]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [16]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

In [17]:
slice

slice

In [18]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [19]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



In [20]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [21]:
slice(-3, None, None).indices(5)

(2, 5, 1)

In [22]:
'ABCDE'[:10:2]

'ACE'

In [23]:
'ABCDE'[0:5:2]

'ACE'

In [24]:
'ABCDE'[-3:]

'CDE'

In [25]:
'ABCDE'[2:5:1]

'CDE'

### A Slice-Aware __getitem__

In [29]:
import operator 

def __len__(self):
    return len(self._components)

def __getitem__(self, key):
    if isinstance(key, slice):
        cls = type(self)
        return cls(self._components[key])
    index = operator.index(key)
    return self._components[index]

In [30]:
Vector.__len__ = __len__
Vector.__getitem__ = __getitem__

In [31]:
v7 = Vector(range(7))
v7[-1]

6.0

In [32]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [33]:
v7[-1:]

Vector([6.0])

In [34]:
try:
    print(f"{v7[1,2]=}")
except Exception as e:
    print(f"{e=}")

e=TypeError("'tuple' object cannot be interpreted as an integer")


## Vector Take #3: Dynamic Attribute Access

In the evolution from `Vector2d` to `Vector`, we lost the ability to access vector compo‐ nents by name (e.g., `v.x`, `v.y`). We are now dealing with vectors that may have a large number of components. Still, it may be convenient to access the first few components with shortcut letters such as `x`, `y`, `z` instead of `v[0]`, `v[1]`, and `v[2]`.

In [35]:
Vector.__match_args__ = ('x', 'y', 'z', 't')

def __getattr__(self, name):
    cls = type(self)
    try:
        pos = cls.__match_args__.index(name)
    except ValueError:
        pos = -1
    if 0<=pos<len(self._components):
        return self._components[pos]
    msg = f'{cls.__name__!r} object has no attribute {name!r}'
    raise AttributeError(msg)

Vector.__getattr__ = __getattr__

In [36]:
v = Vector(range(10))
v.x

0.0

In [37]:
v.y, v.z, v.t

(1.0, 2.0, 3.0)

In [38]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [39]:
v.x

0.0

In [40]:
v.x = 10
v.x

10

In [41]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [ ]:
def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
        if name in cls.__match_args__:
            error = 'readonly attribute {attr_name!r}'
        elif name.islower():
            error = "can't set attributes 'a' to 'z' in {cls_name!r}"
        else:
            error = ''

        if error:
            msg = error.format(cls_name=cls.__name__, attr_name=name)
            raise AttributeError(msg)
    # super().__setattr__(name, value)
    super(Vector, self).__setattr__(name, value)

Vector.__setattr__ = __setattr__

In [48]:
v = Vector(range(10))
v.x

0.0

In [49]:
v.y, v.z, v.t

(1.0, 2.0, 3.0)

In [51]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [52]:
v.x

0.0

In [54]:
try:
    v.x = 10
    v.x
except Exception as e:
    print(f"{e=}")

e=AttributeError("readonly attribute 'x'")


##  Vector Take #4: Hashing and a Faster `==`

Once more we get to implement a `__hash__` method. Together with the existing `__eq__`, this will make `Vector` instances hashable.

In [57]:
import functools
import operator

def __eq__(self, other):
    return tuple(self) == tuple(other)

def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes, 0)

Vector.__eq__ = __eq__
Vector.__hash__ = __hash__

In [58]:
def __eq__(self, other):
    if len(self) != len(other):
        return False
    for a, b in zip(self, other):
        if a!=b:
            return False
    return True

In [59]:
Vector.__eq__ = __eq__

In [60]:
zip(range(3), 'ABC')

In [61]:
list(zip(range(3), 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [62]:
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]

In [63]:
from itertools import zip_longest

In [64]:
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]

## Vector Take #5: Formatting

The `__format__` method of Vector will resemble that of `Vector2d`, but instead of providing a custom display in polar coordinates, `Vector` will use spherical coordi‐nates—also known as “hyperspherical” coordinates, because now we support _n_ dimensions, and spheres are “hyperspheres” in 4D and beyond.6 Accordingly, we’ll change the custom format suffix from `'p'` to `'h'`.

In [65]:
def angle(self, n):
    r = math.hypot(*self[n:])
    a = math.atan2(r, self[n-1])
    if (n == len(self) - 1) and (self[-1] < 0):
        return math.pi * 2 - a
    else:
        return a

def angles(self):
    return (self.angle(n) for n in range(1, len(self)))

def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('h'): # hyperspherical coordinates
        fmt_spec = fmt_spec[:-1]
        coords = itertools.chain([abs(self)],
                                 self.angles())
        outer_fmt = '<{}>'
    else:
        coords = self
        outer_fmt = '({})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(', '.join(components))

@classmethod
def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode) 
    return cls(memv)

Vector.angle = angle
Vector.angles = angles
Vector.__format__ = __format__
Vector.frombytes = frombytes

In [66]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [67]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [68]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [69]:
v1 = Vector([3, 4])

In [71]:
x, y = v1
x, y

(3.0, 4.0)

In [72]:
v1

Vector([3.0, 4.0])

In [74]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [75]:
print(v1)

(3.0, 4.0)


In [76]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [77]:
abs(v1)

5.0

In [78]:
bool(v1), bool(Vector([0, 0]))

(True, False)

In [80]:
v1_clone = Vector.frombytes(bytes(v1))
v1_clone

Vector([3.0, 4.0])

In [81]:
v1 == v1_clone

True

In [83]:
v1 = Vector([3, 4, 5])
x, y, z = v1
x, y, z

(3.0, 4.0, 5.0)

In [84]:
v1

Vector([3.0, 4.0, 5.0])

In [85]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [86]:
print(v1)

(3.0, 4.0, 5.0)


In [87]:
abs(v1)

7.0710678118654755

In [88]:
bool(v1), bool(Vector([0, 0, 0]))

(True, False)

In [89]:
v7 = Vector(range(7))
v7

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [90]:
abs(v7)  # doctest:+ELLIPSIS

9.539392014169456

In [92]:
v1 = Vector([3, 4, 5])
v1_clone = Vector.frombytes(bytes(v1))
v1_clone

Vector([3.0, 4.0, 5.0])

In [93]:
v1 == v1_clone

True

In [94]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [95]:
v1[0], v1[len(v1)-1], v1[-1]

(3.0, 5.0, 5.0)

In [96]:
v7 = Vector(range(7))
v7[-1]

6.0

In [97]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [98]:
v7[-1:]

Vector([6.0])

In [99]:
try:
    print(f"{v7[1,2]=}")
except Exception as e:
    print(f"{e=}")

e=TypeError("'tuple' object cannot be interpreted as an integer")


In [100]:
v7 = Vector(range(10))
v7.x

0.0

In [101]:
v7.y, v7.z, v7.t

(1.0, 2.0, 3.0)

In [102]:
try:
    print(f"{v7.k=}")
except Exception as e:
    print(f"{e=}")

e=AttributeError("'Vector' object has no attribute 'k'")


In [103]:
try:
    v3 = Vector(range(3))
    print(f"{v3.t=}")
except Exception as e:
    print(f"{e=}") 

e=AttributeError("'Vector' object has no attribute 't'")


In [104]:
try:
    v3 = Vector(range(3))
    print(f"{v3.spam=}")
except Exception as e:
    print(f"{e=}") 

e=AttributeError("'Vector' object has no attribute 'spam'")


In [105]:
v1 = Vector([3, 4])
v2 = Vector([3.1, 4.2])
v3 = Vector([3, 4, 5])
v6 = Vector(range(6))

hash(v1), hash(v3), hash(v6)

(7, 2, 1)

In [106]:
import sys
hash(v2) == (384307168202284039 if sys.maxsize > 2**32 else 357915986)

True

In [107]:
v1 = Vector([3, 4])
format(v1)

'(3.0, 4.0)'

In [108]:
format(v1, '.2f')

'(3.00, 4.00)'

In [109]:
format(v1, '.3e')

'(3.000e+00, 4.000e+00)'

In [110]:
v3 = Vector([3, 4, 5])
format(v3)

'(3.0, 4.0, 5.0)'

In [111]:
format(Vector(range(7)))

'(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)'

In [113]:
import itertools

format(Vector([1, 1]), 'h')  # doctest:+ELLIPSIS

'<1.4142135623730951, 0.7853981633974483>'

In [114]:
format(Vector([1, 1]), '.3eh')

'<1.414e+00, 7.854e-01>'

In [115]:
format(Vector([1, 1]), '0.5fh')

'<1.41421, 0.78540>'

In [116]:
format(Vector([1, 1, 1]), 'h')  # doctest:+ELLIPSIS

'<1.7320508075688772, 0.9553166181245093, 0.7853981633974483>'

In [117]:
format(Vector([2, 2, 2]), '.3eh')

'<3.464e+00, 9.553e-01, 7.854e-01>'

In [118]:
format(Vector([0, 0, 0]), '0.5fh')

'<0.00000, 0.00000, 0.00000>'

In [119]:
format(Vector([-1, -1, -1, -1]), 'h')  # doctest:+ELLIPSIS

'<2.0, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'

In [120]:
format(Vector([2, 2, 2, 2]), '.3eh')

'<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>'

In [121]:
format(Vector([0, 1, 0, 0]), '0.5fh')

'<1.00000, 1.57080, 0.00000, 0.00000>'

In [30]:
import sys

In [31]:
%%timeit

i: int = 1_000_000_000
t: float = 0
while i>=0:
    try:
        t += 1/i
    except ZeroDivisionError as e:
        print(f"{e=}", file=sys.stderr)
    i -=1


e=ZeroDivisionError('division by zero')
e=ZeroDivisionError('division by zero')
e=ZeroDivisionError('division by zero')
e=ZeroDivisionError('division by zero')
e=ZeroDivisionError('division by zero')
e=ZeroDivisionError('division by zero')
e=ZeroDivisionError('division by zero')


2min 44s ± 21.7 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


e=ZeroDivisionError('division by zero')


In [33]:
print(f"Total: {t} (with {i=})")

NameError: name 't' is not defined

In [34]:
%%timeit
i: int = 1_000_000_000
t: float = 0
while i>=0:
    if i!=0:
        t += 1/i
    else:
        print(f"Division by zero!", file=sys.stderr)
    i -=1


Division by zero!
Division by zero!
Division by zero!
Division by zero!
Division by zero!
Division by zero!
Division by zero!


3min 5s ± 37.5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Division by zero!


In [35]:
print(f"Total: {t} (with {i=})")

NameError: name 't' is not defined

In [36]:
i: int = 1_000_000
t: float = 0
while i>=0:
    try:
        t += 1/i
    except ZeroDivisionError as e:
        print(f"{e=}", file=sys.stderr)
    i -=1
print(f"Total: {t} (with {i=})")

Total: 14.392726722865772 (with i=-1)


e=ZeroDivisionError('division by zero')


In [ ]:
import time

start_time = time.perf_counter()
block: int = 3

for _ in range(block):
    i: int = 1_000_000_000
    t: float = 0
    while i>=0:
        if i!=0:
            t += 1/i
        else:
            print(f"Division by zero!", file=sys.stderr)
        i -=1
end_time = time.perf_counter()
print(f"Total: {t} (with {i=})")

print(f"IF/ELSE Time taken (time.time()): {end_time-start_time:.2f}s")

start_time = time.perf_counter()
block: int = 3

for _ in range(block):
    i: int = 1_000_000
    t: float = 0
    while i>=0:
        try:
            t += 1/i
        except ZeroDivisionError as e:
            print(f"{e=}", file=sys.stderr)
        i -=1
end_time = time.perf_counter()
print(f"Total: {t} (with {i=})")

print(f"TRY/EXCEPT Time taken (time.time()): {end_time-start_time:.2f}s")


In [38]:
block: int = 3
for _ in range(block):
    t: float = 0
    start_time = time.perf_counter()
    print(f"Loop pass {_}/{block}:")
    for i in range(1_000_000_000,0,-1):
        if i!=0:
            t += 1/i
        else:
            print(f"Division by zero!", file=sys.stderr)
    end_time = time.perf_counter()
    print(f"IF/ELSE Time taken (time.time()): {end_time-start_time:.2f}s")
    print(f"Total: {t} (with {i=})")

for _ in range(block):
    t: float = 0
    start_time = time.perf_counter()
    print(f"Loop pass {_}/{block}:")
    for i in range(1_000_000_000,0,-1):
        try:
            t += 1/i
        except Exception as e:
            print(f"{e=}")
    end_time = time.perf_counter()
    print(f"TRY/EXCEPT Time taken (time.time()): {end_time-start_time:.2f}s")
    print(f"Total: {t} (with {i=})")

Loop pass 0/3:
IF/ELSE Time taken (time.time()): 206.06s
Total: 21.30048150234615 (with i=1)
Loop pass 1/3:
IF/ELSE Time taken (time.time()): 226.18s
Total: 21.30048150234615 (with i=1)
Loop pass 2/3:
IF/ELSE Time taken (time.time()): 255.12s
Total: 21.30048150234615 (with i=1)
Loop pass 0/3:
TRY/EXCEPT Time taken (time.time()): 196.07s
Total: 21.30048150234615 (with i=1)
Loop pass 1/3:
TRY/EXCEPT Time taken (time.time()): 183.71s
Total: 21.30048150234615 (with i=1)
Loop pass 2/3:
TRY/EXCEPT Time taken (time.time()): 184.40s
Total: 21.30048150234615 (with i=1)
